In [1]:
! pip install pandas numpy


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\Khelan Modi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import json
# Load the CSV file
df = pd.read_csv('./data/Food Ingredients and Recipe Dataset with Image Name Mapping.csv')

# Select the first 5000 data points
df = df.head(5000)

# Select only the required columns and add a Price column with dummy data
df = df[['Title', 'Ingredients']]
df['Price'] = np.round(np.random.uniform(5, 20, size=len(df)), 2)  # Round price to 2 decimal places

# Convert DataFrame to JSON, ensure UTF-8 encoding to handle Unicode characters properly
json_data = json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

# Write the JSON data to a file
with open('./data/output_data.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

print("Data has been successfully converted and stored in 'output_data.json'.")


Data has been successfully converted and stored in 'output_data.json'.


Langchain documentation code testing

In [ ]:
from langchain.community.vectorstores.azure_cosmosdb import AzureCosmosDBVectorStore, AzureCosmosDBSimilarityType
from langchain.core.prompts import ChatPromptTemplate
from langchain.openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains.combine_documents import createStuffDocumentsChain
from langchain.chains.retrieval import createRetrievalChain
from langchain.document_loaders.fs.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents from file
loader = TextLoader("./data/state_of_the_union.txt")
raw_documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunkSize=1000, chunkOverlap=0)
documents = splitter.splitDocuments(raw_documents)

# Create Azure Cosmos DB vector store
store = AzureCosmosDBVectorStore.fromDocuments(
  documents,
  OpenAIEmbeddings(),
  databaseName="langchain",
  collectionName="documents"
)

# Create the index
numLists = 100
dimensions = 1536
similarity = AzureCosmosDBSimilarityType.COS
store.createIndex(numLists, dimensions, similarity)

# Performs a similarity search
result_documents = store.similaritySearch(
  "What did the president say about Ketanji Brown Jackson?"
)

print("Similarity search results:")
print(result_documents[0].pageContent)
'''
  Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

  Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

  One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

  And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
'''

# Use the store as part of a chain
model = ChatOpenAI(model="gpt-3.5-turbo-1106")
question_answering_prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "Answer the user's questions based on the below context:\n\n{context}",
  ],
  ["human", "{input}"],
])

combine_docs_chain = createStuffDocumentsChain(llm=model, prompt=question_answering_prompt)

chain = createRetrievalChain(retriever=store.asRetriever(), combineDocsChain=combine_docs_chain)

res = chain.invoke(input="What is the president's top priority regarding prices?")

print("Chain response:")
print(res.answer)
'''
  The president's top priority is getting prices under control.
'''

# Clean up
store.delete()
store.close()


ModuleNotFoundError: No module named 'langchain.community'